In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver_manager
!pip install beautifulsoup4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 24.4 MB/s eta 0:00:00a 0:00:01
zsh:1: command not found: apt-get
zsh:1: command not found: apt
cp: /usr/lib/chromium-browser/chromedriver: No such file or directory


In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager

options = ChromeOptions()
#user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36"
#options.add_argument('user-agent=' + user_agent)
options.add_argument("lang=ko_KR")
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("--no-sandbox")
        
# 크롬 드라이버 최신 버전 설정
service = ChromeService(executable_path=ChromeDriverManager().install(), options=options)

# chrome driver
driver = webdriver.Chrome(service=service, options=options) # <- options로 변경

In [31]:
import time
from bs4 import BeautifulSoup
import csv


# 크롤링할 사이트 URL
url = "https://career.rememberapp.co.kr/job/board/hiring-postings?seed=92987327&search=%7B%22leaderPosition%22%3Afalse%2C%22organizationType%22%3A%22all%22%2C%22applicationType%22%3A%22all%22%2C%22keywords%22%3A%5B%22%EB%8D%B0%EC%9D%B4%ED%84%B0%22%5D%7D"
driver.get(url)

# 페이지 로딩 대기
time.sleep(3)

# 페이지의 HTML 내용 가져오기
page_source = driver.page_source

# BeautifulSoup로 HTML 파싱
soup = BeautifulSoup(page_source, 'html.parser')

# 페이지 소스 가져오기
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 공고 정보 추출
job_list = soup.find_all('li', class_='sc-45abcaef-0')

# 추출한 데이터 저장을 위한 리스트
jobs = []

base_url = "https://career.rememberapp.co.kr"

for job in job_list:
    # 공고제목
    title = job.find('div', class_='sc-45abcaef-6').text.strip() if job.find('div', class_='sc-45abcaef-6') else 'null'
    
    # 회사이름
    company_name = job.find('div', class_='sc-45abcaef-7').text.strip() if job.find('div', class_='sc-45abcaef-7') else 'null'
    
    #디테일 페이지로 가는 주소
    detail_url = job.find('a', class_='sc-567718ed-0 jzTeOc')['href'] if job.find('a', class_='sc-567718ed-0 jzTeOc') else ''
    detail_url = base_url + detail_url if detail_url else ''
    # 등록일
    end_date = job.find('div', class_='sc-db17a579-0').text.strip() if job.find('div', class_='sc-db17a579-0') else 'null'
    
    # 출처 (URL)
    platform_name = 'remember'

    # 데이터 리스트에 추가
    jobs.append({
        'title': title,
        'company_name': company_name,
        'detail_url': detail_url,
        'end_date': end_date,
        'platform_name': platform_name
    })

# 크롤링한 데이터 출력
for job in jobs:
    print(job)

# 드라이버 종료
driver.quit()
# CSV 파일로 저장
csv_file = "/Users/youyoungcheon/Desktop/django-project/crawling/remember_job_list.csv"
csv_columns = ['title', 'company_name', 'detail_url', 'end_date', 'platform_name']

try:
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for job in jobs:
            writer.writerow(job)
    print(f"CSV 파일이 성공적으로 저장되었습니다: {csv_file}")
except IOError:
    print("CSV 파일을 저장하는 중에 오류가 발생했습니다.")

{'title': '[Publishing Platform Div.] Data Infra Engineer (10년 이상)', 'company_name': '(주)크래프톤', 'detail_url': 'https://career.rememberapp.co.kr/job/postings/191174?curationLocation=job_posting_tab&channel=ad_job_postings&jdViewSource=ad_job_postings', 'end_date': '채용 시 마감', 'platform_name': 'remember'}
{'title': 'AI Data Scientist', 'company_name': 'SAP코리아(주)', 'detail_url': 'https://career.rememberapp.co.kr/job/postings/190539?curationLocation=job_posting_tab&channel=ad_job_postings&jdViewSource=ad_job_postings', 'end_date': '채용 시 마감', 'platform_name': 'remember'}
{'title': 'Developer for SAP HANA - Cloud Database Federation to Large Data', 'company_name': 'SAP코리아(주)', 'detail_url': 'https://career.rememberapp.co.kr/job/postings/189907?curationLocation=job_posting_tab&channel=ad_job_postings&jdViewSource=ad_job_postings', 'end_date': '채용 시 마감', 'platform_name': 'remember'}
{'title': '스마트팩토리 솔루션/서비스 데이터 아키텍트', 'company_name': '현대오토에버(주)', 'detail_url': 'https://career.rememberapp.co.kr

In [33]:
import requests
from bs4 import BeautifulSoup
import csv

# CSV 파일 작성 준비
with open('/Users/youyoungcheon/Desktop/django-project/crawling/peoplenjob_job_list.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # CSV 파일 헤더 작성
    writer.writerow(['title', 'company_name', 'detail_url', 'end_date', 'platform_name'])

    # 여러 페이지 크롤링
    for page in range(1, 28):  # 페이지 번호는 원하는 만큼 조정 (예: 1~5페이지)
        url = f'https://www.peoplenjob.com/jobs?field=all&q=%EB%8D%B0%EC%9D%B4%ED%84%B0&page={page}'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        rows = soup.find_all('tr')

        # 각 공고에서 필요한 정보 추출 및 CSV에 기록
        for row in rows:
            # 공고 제목
            title_tag = row.find('td', class_='job-title')
            title = title_tag.find('a').text.strip() if title_tag else 'N/A'

            # 회사 이름
            company_tag = row.find('td', class_='name')
            company_name = company_tag.find('a').text.strip() if company_tag else 'N/A'

            # 상세 페이지 URL
            detail_url_tag = title_tag.find('a') if title_tag else None
            detail_url = f"{detail_url_tag['href']}" if detail_url_tag else 'N/A'

            # 마감일
            end_date_tag = row.find('span', class_='job-fin-date')
            end_date = end_date_tag.text.strip() if end_date_tag else 'N/A'

            # 출처 (플랫폼 이름)
            platform_name = 'peoplenjob'

            # CSV 파일에 작성
            writer.writerow([title, company_name, detail_url, end_date, platform_name])

        print(f"{page} 페이지 크롤링 완료")

print("CSV 파일 저장 완료: 'peoplenjob_job_list.csv'")


1 페이지 크롤링 완료
2 페이지 크롤링 완료
3 페이지 크롤링 완료
4 페이지 크롤링 완료
5 페이지 크롤링 완료
6 페이지 크롤링 완료
7 페이지 크롤링 완료
8 페이지 크롤링 완료
9 페이지 크롤링 완료
10 페이지 크롤링 완료
11 페이지 크롤링 완료
12 페이지 크롤링 완료
13 페이지 크롤링 완료
14 페이지 크롤링 완료
15 페이지 크롤링 완료
16 페이지 크롤링 완료
17 페이지 크롤링 완료
18 페이지 크롤링 완료
19 페이지 크롤링 완료
20 페이지 크롤링 완료
21 페이지 크롤링 완료
22 페이지 크롤링 완료
23 페이지 크롤링 완료
24 페이지 크롤링 완료
25 페이지 크롤링 완료
26 페이지 크롤링 완료
27 페이지 크롤링 완료
CSV 파일 저장 완료: 'peoplenjob_job_list.csv'
